# Exercice 2. Detection d'objets en tentant sa chance ...

Dans cette partie, nous allons générer aléatoirement une série de régions. Pour chaque région, nous envisagerons de la faire passer par le classificateur d'objets entraîné (mono- ou multi-catégories) dans l'exercice **1**. Il n'est pas nécessaire de réentraîner le réseau. Effectuez simplement une prediction de la présence ou l'absence d'un objet dans la regions.

Lors de la génération aléatoire des régions, vous pouvez demander la génération de plusieurs types de regions (rectangles debouts, carrés, rectangles couchés). Vous obtiendrez ainsi pour chaque image un ensemble de regions caractérisées par: (x<sub>i</sub>,y<sub>i</sub>,w<sub>i</sub>,h<sub>i</sub>).

**Q1** Ecrire le code de la function qui génére les regions candidates:

In [1]:
import os
import csv
import cv2
import numpy as np
import math
import random

In [2]:

def preselectRegions(image, regions, classifier, input_size=(128,128), threshold=0.5):
    """
    Préselectionne les régions potentiellement contenant un objet.

    Args:
        image: image originale (H,W,3)
        regions: liste de tuples (x,y,w,h)
        classifier: modèle CNN entraîné (binaire ou multiclasse)
        input_size: taille à laquelle redimensionner les régions avant prediction
        threshold: seuil de confiance pour garder une région (pour binaire)

    Returns:
        preselected: liste de [[x, y, w, h, c, logit], ...]
    """
    preselected = []

    for (x, y, w, h) in regions:
        # Extraire la région et redimensionner
        region_crop = image[y:y+h, x:x+w]
        region_resized = cv2.resize(region_crop, input_size)
        region_resized = np.expand_dims(region_resized, axis=0).astype(np.float32)

        # Normalisation si nécessaire (MobileNet)
        if hasattr(classifier.layers[1], 'weights'):  # détection MobileNet
            region_resized = preprocess_input(region_resized)

        # Prédiction
        logits = classifier(region_resized, training=False).numpy()[0]

        if logits.size == 1:
            # Binaire
            prob = logits[0]
            if prob >= threshold:
                preselected.append([x, y, w, h, 1, prob])
        else:
            # Multiclasse
            c = int(np.argmax(logits))
            logit_c = logits[c]
            preselected.append([x, y, w, h, c, logit_c])

    return preselected


In [3]:
def generate_random_regions(img_shape, n_regions=10, min_size_ratio=0.1, max_size_ratio=0.5):
    H, W = img_shape[:2]
    regions = []

    for _ in range(n_regions):
        # choisir le type de région : carré, rectangle debout, rectangle couché
        region_type = random.choice(['square', 'portrait', 'landscape'])
        
        # largeur et hauteur aléatoires
        size_ratio = random.uniform(min_size_ratio, max_size_ratio)
        if region_type == 'square':
            w = h = int(size_ratio * min(W, H))
        elif region_type == 'portrait':  # rectangle debout
            w = int(size_ratio * W * 0.6)
            h = int(size_ratio * H * 1.2)
        else:  # 'landscape' rectangle couché
            w = int(size_ratio * W * 1.2)
            h = int(size_ratio * H * 0.6)
        
        # limiter la taille pour rester dans l'image
        w = min(w, W)
        h = min(h, H)
        
        # position aléatoire
        x = random.randint(0, W - w)
        y = random.randint(0, H - h)
        
        regions.append((x, y, w, h))
    
    return regions


Vous passerez chacune de ces regions par le réseau (de l'exercice **1**) et vous conserverez uniquement celles pour lesquelles un objet a été reconnu.

Conserver la classe et la valeur du logit correspondant qui nous servira à évaluer la confiance en la detection. Ainsi, pour chaque image vous avez une nouvelle description des regions (x<sub>i</sub>,y<sub>i</sub>,w<sub>i</sub>,h<sub>i</sub>,c<sub>i</sub>,logit<sub>i</sub>) où c<sub>i</sub> correspond à la classe et logit<sub>i</sub> au logit ayant déterminé la classe (il jouera le rôle de la confiance qu'on fait en la detection).

**Q2** Ecrire le code de la fonction qui preselectionne les régions contenant un objet potentiel:

In [4]:
def preselectRegions(image, regions, classifier, input_size=(128,128), threshold=0.5):
    preselected = []

    for (x, y, w, h) in regions:
        # Extraire la région et redimensionner
        region_crop = image[y:y+h, x:x+w]
        region_resized = cv2.resize(region_crop, input_size)
        region_resized = np.expand_dims(region_resized, axis=0).astype(np.float32)

        # Normalisation si nécessaire (MobileNet)
        if hasattr(classifier.layers[1], 'weights'):  # détection MobileNet
            region_resized = preprocess_input(region_resized)

        # Prédiction
        logits = classifier(region_resized, training=False).numpy()[0]

        if logits.size == 1:
            # Binaire
            prob = logits[0]
            if prob >= threshold:
                preselected.append([x, y, w, h, 1, prob])
        else:
            # Multiclasse
            c = int(np.argmax(logits))
            logit_c = logits[c]
            preselected.append([x, y, w, h, c, logit_c])

    return preselected


Pour les régions classées comme contenant un objet, appliquez l'algorithme de suppression des non-maxima classe par classe.

Triez les régions par ordre décroissant de score de confiance/prédiction.

Calculez l'IoU entre la première région de la liste et toutes les autres. Si l'IoU est supérieur au seuil, supprimez la région correspondante de la liste.

Répétez l'opération jusqu'à ce que toutes les régions aient été traitées ou supprimées.

**Q3** Proposez une implementation pour le calcul de l'IoU et pour l'algorithm de NonMaxSuppression

In [5]:
def IoU(regionA, regionB):
    xA = max(regionA[0], regionB[0])
    yA = max(regionA[1], regionB[1])
    xB = min(regionA[0] + regionA[2], regionB[0] + regionB[2])
    yB = min(regionA[1] + regionA[3], regionB[1] + regionB[3])

    # intersection width/height
    inter_w = max(0, xB - xA)
    inter_h = max(0, yB - yA)
    inter_area = inter_w * inter_h

    # union
    areaA = regionA[2] * regionA[3]
    areaB = regionB[2] * regionB[3]
    union_area = areaA + areaB - inter_area

    if union_area == 0:
        return 0
    else:
        return inter_area / union_area

        
def nonmaxSuppression(image, classes, regionsWithClassAndLogit, iou_threshold=0.5):
    selected_regions = []

    for cls in classes:
        # extraire les régions de cette classe
        cls_regions = [r for r in regionsWithClassAndLogit if r[4] == cls]

        # trier par logit décroissant
        cls_regions = sorted(cls_regions, key=lambda r: r[5], reverse=True)

        keep = []
        while cls_regions:
            # prendre la première région
            current = cls_regions.pop(0)
            keep.append(current)

            # supprimer toutes les régions qui se chevauchent trop
            cls_regions = [r for r in cls_regions if IoU(current, r) <= iou_threshold]

        selected_regions.extend(keep)

    return selected_regions


**Q4** Calculez ensuite, pour chaque région restante, l'IoU avec les boîtes englobantes consituant la vérité de terrain et gardez uniquement les fenetre avec un IoU supperieur à un seuil fixé. Parcourez les regions en fonction des leur niveau de confiance. Au fur et à mesure que les groundtruth_regions se voient attribuer une candidate_region, il faut enlever cette groundtruth_region de la liste afin d'éviter que plusieurs regions candidates soient ratachées à la même région de la groundtruth.

In [6]:
def compareToGroundTruth(image, groundtruth_regions, candidate_regions, iou_threshold=0.5):
    # Trier les candidates par logit décroissant
    candidates_sorted = sorted(candidate_regions, key=lambda r: r[5], reverse=True)

    TPRegions = []
    FNDetected = []
    # Faire une copie des ground-truth pour enlever au fur et à mesure
    remaining_gt = groundtruth_regions.copy()

    for cand in candidates_sorted:
        matched = False
        best_iou = 0
        best_gt = None
        for gt in remaining_gt:
            iou = IoU(cand, gt)
            if iou > best_iou:
                best_iou = iou
                best_gt = gt

        if best_iou >= iou_threshold:
            TPRegions.append(cand)
            remaining_gt.remove(best_gt)
            matched = True
        else:
            FNDetected.append(cand)

    # Les gt restant sont les FN non détectés
    FNNonDetected = remaining_gt

    return TPRegions, FNDetected, FNNonDetected


## Détection d'une classe d'objets

Appliquez la procédure à l'ensemble d'images contenant l'étiquette 4 (Éléphant). 
Nous ne découperons plus par rapport à l'éléphant, mais considérons l'image dans son ensemble. 
Dans cette situation, nous considérons que tout autre objet apparaîssant sur les images que l'éléphant fait partie de l'arrière plan. 
Lors du chargement des données, uniquement les regions concernant l'éléphant feront partie de la vérité de terrain.

Utilisez un classifier binaire capable de reconnaître un éléphant.

Vous devez coder une nouvelle version de la fonction `load_elephant_objects` afin de prendre en compte ces considérations.

**Q5** Coder `load_elephant_objects`

In [7]:
def load_elephant_objects(imgs_path, max_samples=None):
    x = []
    y = []
    count_samples = 0

    if max_samples is None:
        max_samples = np.iinfo(np.int32).max  # valeur maximale par défaut

    imgs_files = os.listdir(imgs_path + "images/")
    for i, img_file in enumerate(imgs_files):
        if count_samples >= max_samples:
            break

        label_file = img_file[:-4] + ".txt"
        img_init = cv2.imread(imgs_path + "images/" + img_file)
        if img_init is None:
            continue

        labels = csv.reader(open(imgs_path + "labels/" + label_file, "r"), delimiter=' ')
        rows = list(labels)

        elephant_boxes = []
        for row in rows:
            if row[0] != '4':  # garder uniquement les éléphants
                continue

            bbox = np.array(row[1:], dtype=np.float32)
            w = int(bbox[2] * img_init.shape[1])
            h = int(bbox[3] * img_init.shape[0])
            x0 = max(0, int(bbox[0] * img_init.shape[1] - w / 2))
            y0 = max(0, int(bbox[1] * img_init.shape[0] - h / 2))
            x1 = min(img_init.shape[1], int(x0 + w))
            y1 = min(img_init.shape[0], int(y0 + h))

            elephant_boxes.append([x0, y0, x1 - x0, y1 - y0])

        if elephant_boxes:
            x.append(img_init)
            y.append(elephant_boxes)
            count_samples += 1

    return x, y


In [8]:
data_path = "../data/toy-dataset"
train_path = os.path.join(data_path, "train/")
test_path = os.path.join(data_path, "test/")

In [9]:
# Charger les images de test contenant des éléphants
x_test, y_test = load_elephant_objects(test_path)  # test_path = "../data/toy-dataset/test/"

print("Nombre d'images test contenant des éléphants :", len(x_test))
for i, boxes in enumerate(y_test[:5]):
    print(f"Image {i} : {len(boxes)} éléphant(s) - boxes :", boxes)


Nombre d'images test contenant des éléphants : 30
Image 0 : 1 éléphant(s) - boxes : [[307, 34, 52, 62]]
Image 1 : 1 éléphant(s) - boxes : [[316, 43, 40, 90]]
Image 2 : 1 éléphant(s) - boxes : [[137, 31, 43, 73]]
Image 3 : 1 éléphant(s) - boxes : [[285, 150, 35, 88]]
Image 4 : 1 éléphant(s) - boxes : [[188, 193, 45, 89]]


**Q6** Pour chaque image du test set affichez le nombre de TP, FP, TN.
Choissisez un sous-ensemble de 3 images sur lesquelles vous superposer en vert la GT, en blue les TP, en orange les FP et en rouge les FN.

In [10]:
def evaluate_image(image, groundtruth_boxes, candidate_regions, iou_threshold=0.5):
    TP, FP, FN = compareToGroundTruth(image, groundtruth_boxes, candidate_regions, iou_threshold)
    
    # Pour TN : dans notre cas, toutes les régions candidates non détectées et qui ne contiennent pas d'éléphant
    TNCount = 0  # si on ne génère pas explicitement les candidates négatives, TN = 0
    return TP, FP, FN, TNCount


In [11]:
def plot_image_with_gt_tp_fp_fn(image, groundtruth_boxes, TPRegions, FPRegions, FNNonDetected):
    img_disp = image.copy()

    # Ground Truth - vert
    for r in groundtruth_boxes:
        x, y, w, h = r[:4]
        cv2.rectangle(img_disp, (x,y), (x+w, y+h), (0,255,0), 2)
    
    # True Positives - bleu
    for r in TPRegions:
        x, y, w, h = r[:4]
        cv2.rectangle(img_disp, (x,y), (x+w, y+h), (0,0,255), 2)

    # False Positives - orange
    for r in FPRegions:
        x, y, w, h = r[:4]
        cv2.rectangle(img_disp, (x,y), (x+w, y+h), (255,165,0), 2)

    # False Negatives - rouge
    for r in FNNonDetected:
        x, y, w, h = r[:4]
        cv2.rectangle(img_disp, (x,y), (x+w, y+h), (255,0,0), 2)

    plt.figure(figsize=(10,8))
    plt.imshow(cv2.cvtColor(img_disp, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()


In [18]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet import preprocess_input

# Exemple : chemin vers le modèle sauvegardé
model_binary = load_model("model_elephant_binary.keras")

In [21]:
def resize(imgs, target_resize=(128, 128)):
    resized_imgs = []
    for img in imgs:
        img_resized = cv2.resize(img, target_resize)
        resized_imgs.append(img_resized)
    return np.array(resized_imgs)

x_test = tf.stack(x_test)
y_test = np.array(y_test)

In [22]:
iou_threshold = 0.5
results = []

for img, gt_boxes in zip(x_test, y_test):
    # 1. Générer des régions candidates
    regions = generate_random_regions(img.shape, n_regions=20)
    
    # 2. Préselection par CNN binaire
    candidate_regions = preselectRegions(img, regions, model_binary, input_size=(224,224), threshold=0.5)
    
    # 3. NMS
    classes_present = [1]  # binaire : 1 = éléphant
    candidate_regions_nms = nonmaxSuppression(img, classes_present, candidate_regions, iou_threshold=0.5)
    
    # 4. Évaluation vs GT
    TP, FP, FN = compareToGroundTruth(img, gt_boxes, candidate_regions_nms, iou_threshold=iou_threshold)
    TN = 0  # on ne génère pas explicitement les TN ici
    results.append([len(TP), len(FP), TN, len(FN)])

# Affichage des résultats
for i, (TPc, FPc, TNc, FNc) in enumerate(results):
    print(f"Image {i}: TP={TPc}, FP={FPc}, TN={TNc}, FN={FNc}")


error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'resize'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [ ]:
for i in range(3):
    img = x_test[i]
    gt_boxes = y_test[i]

    # Générer et filtrer les régions pour la visualisation
    regions = generate_random_regions(img.shape, n_regions=20)
    candidate_regions = preselectRegions(img, regions, model_binary, input_size=(224,224), threshold=0.5)
    candidate_regions_nms = nonmaxSuppression(img, [1], candidate_regions, iou_threshold=0.5)
    TP, FP, FN = compareToGroundTruth(img, gt_boxes, candidate_regions_nms, iou_threshold=iou_threshold)

    plot_image_with_gt_tp_fp_fn(img, gt_boxes, TP, FP, FN)


**Q7** Calculez la précision moyenne (AP) pour les images relevant de cette étiquette sur le test set pour un IoU de 0.5.

Afin de calculer la précision moyenne, il est nécessaire de traiter l'ensemble des detections dans leur globalité par rapport à leur niveau de confiance.

Regroupez dans une liste l'ensemble de regions candidates issues de **Q4** tout en conservant une information relative à leur appartenance aux ensembles TPregions ou FPregions. Triez cette liste en fonction de niveau de confiance et parcourez la de manière décroissante.

Initialement le rappel est à zero car TP=0, FP=0 et FN=#total regions issues de la **Q4**
Au fur et à mesure qu'on traite des regions de la liste on met à jour les valeurs cumulées TP et FP pour calculer la précion et le rappel à chaque nouvelle detection.

Tracez un graphique des points (précision, rappel) intermediaires.

Vous pouvez utiliser la fonction `sklearn.metrics.auc` pour calculer la valeur d'AUC en fournissant les listes avec les valeurs intermediaires de (précision et rappel).

## Détection de plusieurs classes

Appliquez la procédure à l'ensemble d'images du dataset. 
Lors du chargement des données, l'ensemble des regions concernant des objets feront partie de la vérité de terrain. Cette fois-ci l'on gardera également des informations sur la classe spécifique associée à une boîte englobante.

**Q7** Vous devez coder une nouvelle version de la fonction `load_objects` afin de prendre en compte ces considérations.

In [ ]:
def load_elephant_objects():
    ...
    return x,y #x ensemble d'images contenant des éléphants, y ensemble de positions des boîtes englobantes
    #attention x=[img1,img1,img1,...], y=[[r11,r12],[r21],[r21,r22,r23],...] 
    #si l'img1 contient deux objets situés à r11 et r12
    #si l'img2 contient un seul objet
    #si l'img3 contient trois objets
    #rij = [xij,yij,wij,hij,cij] où cij correspond à la classe de l'objet

**Q8** Calculez la précision moyenne (AP) pour l'ensemble de données pour un IoU de 0.5.
Par rapport au cas mono-classe, il est d'usage de calculer l'AUC classe par classe (et donc considerer les régions classe par classe) avant de calculer une moyenne qui tient compte de nombre d'instance de chaque classe.


**Q9** (optionnel) Pour guider mieux le processus de génération de regions candidates, vous pouvez également orienter le processus aléatoire en considérant les régions de l'image où se trouvent beaucoup de points caractéristiques (Harris, SIFTs, etc.).
Est-ce que cela améliore les résultats ?
